<a href="https://colab.research.google.com/github/Stephane-Masamba/classification-predict-streamlit-template/blob/Mikael/Mikael_Daniels_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install spacy
!pip install sklearn

In [61]:
import spacy.cli
spacy.cli.download('en_core_web_sm')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
import spacy

In [0]:
import re

In [0]:
nlp = spacy.load('en_core_web_sm')

In [0]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB

In [66]:
train = pd.read_csv('train (1).csv')
print(train.head())

   sentiment                                            message  tweetid
0          1  PolySciMajor EPA chief doesn't think carbon di...   625221
1          1  It's not like we lack evidence of anthropogeni...   126103
2          2  RT @RawStory: Researchers say we have three ye...   698562
3          1  #TodayinMaker# WIRED : 2016 was a pivotal year...   573736
4          1  RT @SoyNovioDeTodas: It's 2016, and a racist, ...   466954


In [67]:
test = pd.read_csv('test (1).csv')
print(test.head())

                                             message  tweetid
0  Europe will now be looking to China to make su...   169760
1  Combine this with the polling of staffers re c...    35326
2  The scary, unimpeachable evidence that climate...   224985
3  @Karoli @morgfair @OsborneInk @dailykos \nPuti...   476263
4  RT @FakeWillMoore: 'Female orgasms cause globa...   872928


In [68]:
sample = pd.read_csv('sample_submission.csv')
print(sample.head())

   tweetid  sentiment
0   169760          1
1    35326          1
2   224985          1
3   476263          1
4   872928          1


In [0]:
i = 0
for tweet in train['message']:
  tweet = tweet.lower()
  tweet = re.sub(r'http\S+', 'LINK', tweet)
  tweet = re.sub(r'@\S+', 'USER_REF', tweet)
  tweet = re.sub(r'[^\w\s]', '', tweet)
  tweet = tweet.lstrip()
  tweet = tweet.rstrip()
  train.loc[i, 'message'] = tweet
  i += 1

In [70]:
print(train.head())

   sentiment                                            message  tweetid
0          1  polyscimajor epa chief doesnt think carbon dio...   625221
1          1  its not like we lack evidence of anthropogenic...   126103
2          2  rt USER_REF researchers say we have three year...   698562
3          1  todayinmaker wired  2016 was a pivotal year in...   573736
4          1  rt USER_REF its 2016 and a racist sexist clima...   466954


In [0]:
j = 0
for message in test['message']:
  message = message.lower()
  message = re.sub(r'http\S+', 'LINK', message)
  message = re.sub(r'@\S+', 'USER_REF', message)
  message = re.sub(r'[^\w\s]', '', message)
  message = message.lstrip()
  message = message.rstrip()
  test.loc[j, 'message'] = message
  j += 1

In [72]:
test.head()

,message,tweetid
0,europe will now be looking to china to make su...,169760
1,combine this with the polling of staffers re c...,35326
2,the scary unimpeachable evidence that climate ...,224985
3,USER_REF USER_REF USER_REF USER_REF \nputin go...,476263
4,rt USER_REF female orgasms cause global warmin...,872928


In [0]:
nlp.Defaults.stop_words.add('LINK')

In [0]:
nlp.Defaults.stop_words.add('USER_REF')

In [75]:
train.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

In [76]:
test.isnull().sum()

message    0
tweetid    0
dtype: int64

In [93]:
X_train = train['message']
X_train

0        polyscimajor epa chief doesnt think carbon dio...
1        its not like we lack evidence of anthropogenic...
2        rt USER_REF researchers say we have three year...
3        todayinmaker wired  2016 was a pivotal year in...
4        rt USER_REF its 2016 and a racist sexist clima...
                               ...                        
15814    rt USER_REF they took down the material on glo...
15815    rt USER_REF how climate change could be breaki...
15816    notiven rt nytimesworld what does trump actual...
15817    rt USER_REF hey liberals the climate change cr...
15818    rt USER_REF USER_REF climate change equation i...
Name: message, Length: 15819, dtype: object

In [94]:
y_train = train['sentiment']
y_train

0        1
1        1
2        2
3        1
4        1
        ..
15814    1
15815    2
15816    0
15817   -1
15818    0
Name: sentiment, Length: 15819, dtype: int64

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tf_vect = TfidfVectorizer()

In [0]:
X_train = tf_vect.fit_transform(X_train)

In [0]:
from sklearn.svm import SVC

In [0]:
sv = SVC()

In [96]:
sv.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [97]:
predictions = sv.predict(X_train)
predictions

array([ 1,  1,  2, ...,  1, -1,  0])

In [0]:
scores = cross_val_score(sv, X_train, y_train, cv=5)

In [102]:
scores

array([0.72945638, 0.73103666, 0.73103666, 0.73198483, 0.73190009])

In [103]:
accuracy = np.mean(scores)
accuracy

0.7310829266810902

In [0]:
def accuracy(model):
     features = train['message']
     target = train['sentiment']

     train_scores = []
     test_scores = []

     tf_vect = TfidfVectorizer()

     folds = KFold(n_splits=5, shuffle=True)

     for train_index, test_index in folds.split(features):
          x_train, x_test = features.iloc[train_index], features.iloc[test_index]    
          y_train, y_test = target.iloc[train_index], target.iloc[test_index]

          x_train = tf_vect.fit_transform(x_train)
          x_test = tf_vect.transform(x_test)
     
          model.fit(x_train, y_train)
          train_predictions = model.predict(x_train)
          test_predictions = model.predict(x_test)

          train_score = accuracy_score(y_train, train_predictions)
          train_scores.append(train_score)

          test_score = accuracy_score(y_test, test_predictions)
          test_scores.append(test_score)

     avg_train_accuracy = np.mean(train_scores)
     avg_test_accuracy = np.mean(test_scores)

     return [avg_train_accuracy, avg_test_accuracy]

In [0]:
sv = SVC()

In [125]:
accuracy(sv)

[0.9517826848469622, 0.734053649718038]